In [1]:
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
from torch.nn import Module, Parameter
from torch import FloatTensor
from scipy import signal
import numpy as np
from torchaudio import transforms
import matplotlib.pyplot as plt
import IPython.display as ipd
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from scipy.signal import sosfiltfilt
import os
dirname = os.path.abspath('')
rootdir = os.path.split(dirname)[0]

H1_TRAINING_INPUT_PATH = "".join([rootdir, "/data/train/ht1-input.wav"])
H1_TRAINING_TARGET_PATH = "".join([rootdir, "/data/train/ht1-target.wav"])

metadata = torchaudio.info(H1_TRAINING_INPUT_PATH)
print(metadata)


AudioMetaData(sample_rate=44100, num_frames=14994001, num_channels=1, bits_per_sample=16, encoding=PCM_S)


In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device=", device) 

device= cpu


In [3]:
train_input, fs = torchaudio.load(H1_TRAINING_INPUT_PATH)
train_target, fs = torchaudio.load(H1_TRAINING_TARGET_PATH)

In [4]:
#len(train_input)
print(train_input.shape)
print(train_target.shape)

torch.Size([1, 14994001])
torch.Size([1, 14994001])


In [5]:
train_input_array = train_input.numpy().reshape(-1)
train_target_array = train_target.numpy().reshape(-1)

# For Reproducibility

In [6]:
torch.manual_seed(0)

## Initialize Dataloader

In [7]:
class DIIRDataSet(Dataset):
    def __init__(self, input, target, sequence_length):
        self.input = input
        self.target = target
        
        self._sequence_length = sequence_length
        self.input_sequence = self.wrap_to_sequences(self.input, self._sequence_length)
        self.target_sequence = self.wrap_to_sequences(self.target, self._sequence_length)
        self._len = self.input_sequence.shape[0]

    def __len__(self):
        return self._len

    def __getitem__(self, index):
        return {'input': self.input_sequence[index, :, :]
               ,'target': self.target_sequence[index, :, :]}

    def wrap_to_sequences(self, data, sequence_length):
        num_sequences = int(np.floor(data.shape[0] / sequence_length))
        print(num_sequences)
        truncated_data = data[0:(num_sequences * sequence_length)]
        wrapped_data = truncated_data.reshape((num_sequences, sequence_length, 1))
        wrapped_data = wrapped_data.permute(0,2,1)
        print(wrapped_data.shape)
        return np.float32(wrapped_data)


In [8]:
train_input.squeeze(0).shape

torch.Size([14994001])

In [9]:
batch_size = 512#1024
sequence_length = 80
train_dataset=DIIRDataSet(train_input.squeeze(0), train_target.squeeze(0), sequence_length)
loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = False, pin_memory=True, drop_last=True) #? what does the shuffle really shuffles here?

187425
torch.Size([187425, 1, 80])
187425
torch.Size([187425, 1, 80])


In [10]:
len(loader)

366

# Declare Model

In [11]:
class IIRNN(Module):
    def __init__(self, n_input=1, n_output=1, hidden_size=80, n_channel=1):
        super(IIRNN, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size = 80, hidden_size = self.hidden_size)

        self.fc1 = nn.Linear(self.hidden_size, 80)
#         self.fc2 = nn.Linear(512, 80)
#         self.fc3 = nn.Linear(80 , 80)

        
        self.mlp_layer = nn.Sequential(
            self.fc1 ,
#             nn.Tanh(),
#             self.fc2,
#             nn.Tanh(),
#             self.fc3
        )

    def forward(self, x):

    
#         print(f"xshape: {x.shape}")
        x, hn = self.lstm(x) # output; (sequence_length, batch_size, hidden_size)
#         print(f"x rnn shape: {x.shape}")
        x = self.mlp_layer(x)
#         x, hn = self.lstm(x)

        return x


In [12]:
model = IIRNN()

## Define optimizer and criterion

In [13]:
import torch.nn as nn
from torch.optim import Adam

n_epochs = 100
lr = 1e-3

optimizer = Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

criterion = nn.MSELoss()

# Define train loop

In [14]:
import torchaudio
from torchaudio.functional import lfilter

In [15]:
def train(criterion, model, loader, optimizer):
    model.train()
    device = next(model.parameters()).device
    total_loss = 0
    
    for ind, batch in enumerate(loader):
        input_seq_batch = batch['input'].to(device)
        target_seq_batch = batch['target'].to(device)

        optimizer.zero_grad()
        predicted_output = model(input_seq_batch)
        
        
        target_seq_batch_filt = lfilter(target_seq_batch, torch.Tensor([1,0]), torch.Tensor([1, -0.95]))
        predicted_output_filt = lfilter(predicted_output, torch.Tensor([1,0]), torch.Tensor([1, -0.95]))
        
        loss = criterion(target_seq_batch_filt, predicted_output_filt)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    total_loss /= len(loader)
    return total_loss

## Train!

In [16]:
for epoch in range(n_epochs):
    loss = train(criterion, model, loader, optimizer)
    print("Epoch {} -- Loss {:3E}".format(epoch, loss))

Epoch 0 -- Loss 5.951266E-03
Epoch 1 -- Loss 5.362060E-03
Epoch 2 -- Loss 5.199638E-03
Epoch 3 -- Loss 5.136401E-03
Epoch 4 -- Loss 5.086308E-03
Epoch 5 -- Loss 5.027675E-03
Epoch 6 -- Loss 4.980498E-03
Epoch 7 -- Loss 4.948835E-03
Epoch 8 -- Loss 4.924637E-03
Epoch 9 -- Loss 4.904404E-03
Epoch 10 -- Loss 4.876002E-03
Epoch 11 -- Loss 4.857945E-03
Epoch 12 -- Loss 4.842755E-03
Epoch 13 -- Loss 4.828449E-03
Epoch 14 -- Loss 4.820232E-03
Epoch 15 -- Loss 4.803886E-03
Epoch 16 -- Loss 4.790378E-03
Epoch 17 -- Loss 4.777608E-03
Epoch 18 -- Loss 4.765093E-03
Epoch 19 -- Loss 4.752421E-03
Epoch 20 -- Loss 4.737420E-03
Epoch 21 -- Loss 4.722519E-03
Epoch 22 -- Loss 4.706834E-03
Epoch 23 -- Loss 4.690356E-03
Epoch 24 -- Loss 4.673349E-03
Epoch 25 -- Loss 4.655986E-03
Epoch 26 -- Loss 4.638289E-03
Epoch 27 -- Loss 4.620135E-03
Epoch 28 -- Loss 4.601429E-03
Epoch 29 -- Loss 4.582297E-03
Epoch 30 -- Loss 4.563038E-03
Epoch 31 -- Loss 4.544078E-03
Epoch 32 -- Loss 4.525789E-03
Epoch 33 -- Loss 4.5

# Evaluate

In [17]:
save_path = os.path.join('../models/lstm_mlp_premp_filter'.format(n_epochs-1))
torch.save(model.state_dict(), save_path)

In [18]:
val_batch_size = 128
sequence_length = 80
val_dataset=DIIRDataSet(train_input.squeeze(0), train_target.squeeze(0), sequence_length)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle = False, pin_memory=True, drop_last=True)

187425
torch.Size([187425, 1, 80])
187425
torch.Size([187425, 1, 80])


In [19]:
def inspect_file(path):
    print("-" * 10)
    print("Source:", path)
    print("-" * 10)
    print(f" - File size: {os.path.getsize(path)} bytes")
    print(f" - {torchaudio.info(path)}")

In [20]:
def save_audio(batch):
    #1024,512,1
    out_batch = batch.detach().cpu()
    out_batch = out_batch.squeeze(-1).flatten()
    print(out_batch.shape)
    return out_batch

In [21]:
import soundfile as sf

out_path = '../output/'
sample_rate = 44100
save_tensor = torch.zeros(14994001,80)
with torch.no_grad():
    for i, val_batch in enumerate(val_loader):
        input_seq_batch = val_batch['input'].to(device)
        #target_seq_batch = val_batch['target'].to(device)
        predicted_output = model(input_seq_batch)
        output_tmp = predicted_output.squeeze().detach().cpu()
        #print(output_tmp.shape)
        save_tensor[i,:] = output_tmp
    
    print(save_tensor.shape)
    out_audio = save_audio(save_tensor)
    print(out_audio.shape)
    path = os.path.join(out_path, "lstm_mlp_premp_filter_h1.wav")
    print("Exporting {}".format(path))
    sf.write(path, out_audio, sample_rate,'PCM_24')
    #torchaudio.save(path, out_audio, sample_rate, encoding="PCM_S", bits_per_sample=16)
    inspect_file(path)
    

torch.Size([14994001, 80])
torch.Size([1199520080])
torch.Size([1199520080])
Exporting ../output/lstm_mlp_premp_filter_h1.wav
----------
Source: ../output/lstm_mlp_premp_filter_h1.wav
----------
 - File size: 3598560284 bytes
 - AudioMetaData(sample_rate=44100, num_frames=1199520080, num_channels=1, bits_per_sample=24, encoding=PCM_S)
